In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [16]:

def string2number(input):
    """" This function takes words One, Two, Three, Four, Five and turns into numbers
    Returns integers only

    Parameters
    ----------
    input : single string input

    Returns
    -------
    int type number

    Notes
    -----
    Amazingly particular function, it doesn't do much but it's honest. 

    Examples
    --------
    string2number('One')
    Output: 1

    """
    texts = {
        'One': 1,
        'Two': 2,
        'Three': 3,
        'Four': 4,
        'Five': 5
    }
    return int(texts.get(input))


with open('scraping_results.csv', mode='w') as csv_file:
    fieldnames = ['Book Title', 'Book Price', 'Book Rating', 'Page Found']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

title = []
price = []
rating = []
num_page = []

url = 'http://books.toscrape.com/catalogue/page-1.html'

def scrapper(web_page, page_number):
    #print(f'scanning page: {web_page} \n-> Parameters: {page_number}, {web_page}')
    html_text = requests.get(web_page).text
    soup = BeautifulSoup(html_text, 'lxml')
    books = soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')
    pages = soup.find('li', class_ = 'current').text.strip()
    total_pages = int(pages[-2:])

    for book in books:
        book_title = book.h3.a['title']
        book_price = book.find('p', class_='price_color').text
        book_rating = book.p['class']
        #print(f'Book Title:{book_title} || Book Price:{book_price[1:]} || Book Rating:{book_rating} || Page found:{page_number}')
        title.append(book_title)
        price.append(book_price[2:])
        rating.append(book_rating)
        num_page.append(page_number)
    #print(f'Page {page_number}, goint to page {page_number+1}')

    if page_number < total_pages:
        page_number += 1
        #print(f'actual page_number {page_number}')
        url = 'http://books.toscrape.com/catalogue/page-' + str(page_number) + '.html'
        scrapper(url, page_number)

scrapper(url, 1)

In [23]:
int_ratings = [string2number(rates[1]) for rates in rating]

data = {'Book-Title': title, 'Book-Price': pd.to_numeric(price), 'Book-Rating': int_ratings, 'Page-Found': num_page}

df = pd.DataFrame(data)


In [24]:
df.to_csv(r'scraping_results2.csv')

In [19]:
df2 = pd.DataFrame(data)

df2.to_xml(r'scraping.xml')

price_mean = df.mean()[0].round(2)

unique_pages = pd.DataFrame(num_page, columns=['Pages']).drop_duplicates()

books_with_the = df.loc[(df['Book Title'].str.contains('^The'))]
books_with_the.head()

book_per_page = df.groupby(['Page Found']) \
                  .agg({'Book Title':'size', 'Book Price':'mean', 'Book Rating':'mean'}) \
                  .rename(columns={'Book Title':'Book per Page','Book Price':'Mean Price', 'Book Rating':'Mean Rating'}) \
                  .round(2)

book_per_page.head()